In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# ---------- helpers ----------
def load_npy(p): 
    return np.asarray(np.load(Path(p), allow_pickle=True))

def zscore(x):
    x = np.asarray(x, float)
    mu, sd = np.mean(x), np.std(x)
    return (x - mu) / (sd if sd > 0 else 1.0)

def collapse_to_1d(x, prefer_time_axis=1, method="rms"):
    """
    Convert radar (range_bins x time) or (time x features) to a 1-D time series.
    - Detects the longest axis as 'time' unless prefer_time_axis matches it.
    - method:
        'rms'  : sqrt(mean(bin^2))
        'mean' : mean across bins
        'max'  : max across bins (good for strong single-return)
        'pca1' : first PC (requires sklearn; skip unless you need it)
    """
    x = np.asarray(x, float)
    if x.ndim == 1:
        return x

    # choose time axis (longest by default)
    lengths = list(x.shape)
    t_ax = prefer_time_axis if lengths[prefer_time_axis] == max(lengths) else int(np.argmax(lengths))
    # move time to axis 0
    x = np.moveaxis(x, t_ax, 0)                       # shape: (T, ...)
    X = x.reshape(x.shape[0], -1)                     # (T, BINS)

    if method == "rms":
        y = np.sqrt(np.mean(X**2, axis=1))
    elif method == "mean":
        y = np.mean(X, axis=1)
    elif method == "max":
        y = np.max(X, axis=1)
    elif method == "pca1":
        from sklearn.decomposition import PCA
        y = PCA(n_components=1).fit_transform(X).ravel()
    else:
        raise ValueError("method must be one of {'rms','mean','max','pca1'}")

    return np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)

def resample_linear(y, fs_in, fs_out):
    if fs_in == fs_out:
        return y
    t_in  = np.arange(len(y)) / fs_in
    t_out = np.arange(0, t_in[-1], 1.0/fs_out)
    return np.interp(t_out, t_in, y)

def xcorr_fft(a, b):
    """full cross-correlation via FFT; returns (corr, lags_in_samples)."""
    n = int(2 ** np.ceil(np.log2(len(a) + len(b) - 1)))
    fa, fb = np.fft.rfft(a, n=n), np.fft.rfft(b, n=n)
    corr = np.fft.irfft(fa * np.conj(fb), n=n)
    full_len = len(a) + len(b) - 1
    corr = corr[:full_len]
    lags = np.arange(-len(b) + 1, len(a))
    return corr, lags

# ---------- main function ----------
def compare_uwb_vs_mmw(
    uwb_path, mmw_path,
    fs_uwb=1000.0, fs_mmw=1000.0,              # put real Hz if you know them
    prefer_time_axis=1,                         # many radar arrays are (range_bins, time) -> time axis often 1
    collapse="rms",                             # 'rms' is robust; try 'max' if returns are sparse
    show=True
):
    uwb_raw = load_npy(uwb_path)
    mmw_raw = load_npy(mmw_path)

    # 1) collapse to 1-D time series
    uwb_1d = collapse_to_1d(uwb_raw, prefer_time_axis=prefer_time_axis, method=collapse)
    mmw_1d = collapse_to_1d(mmw_raw, prefer_time_axis=prefer_time_axis, method=collapse)

    # 2) normalize
    uwb_1d = zscore(uwb_1d)
    mmw_1d = zscore(mmw_1d)

    # 3) resample to common fs (use the higher to preserve information)
    target_fs = max(fs_uwb, fs_mmw)
    uwb_rs = resample_linear(uwb_1d, fs_uwb, target_fs)
    mmw_rs = resample_linear(mmw_1d, fs_mmw, target_fs)

    # 4) equal length
    L = min(len(uwb_rs), len(mmw_rs))
    uwb_rs, mmw_rs = uwb_rs[:L], mmw_rs[:L]

    # 5) cross-correlation
    corr, lags = xcorr_fft(uwb_rs - np.mean(uwb_rs), mmw_rs - np.mean(mmw_rs))
    peak = int(np.argmax(np.abs(corr)))
    lag_samples = int(lags[peak])         # lag of mmWave relative to UWB
    lag_seconds = lag_samples / target_fs # +: mmWave lags; UWB leads

    # quick quality numbers
    # align and compute correlation at zero-lag vs. aligned
    r = np.corrcoef(uwb_rs, mmw_rs)[0,1]
    mmw_shift = np.roll(mmw_rs, -lag_samples)  # shift mmWave left so peaks align
    r_aligned = np.corrcoef(uwb_rs, mmw_shift)[0,1]

    print(f"Estimated lag (mmWave relative to UWB): {lag_samples} samples @ {target_fs:.1f} Hz  →  {lag_seconds:.6f} s")
    print("Sign: positive = mmWave occurs later (UWB leads).")
    print(f"Pearson r (no shift) = {r:.3f} | after alignment = {r_aligned:.3f}")

    if show:
        t = np.arange(L) / target_fs

        # raw signals
        plt.figure(figsize=(10,4))
        plt.plot(t, zscore(uwb_rs), label="UWB (z)")
        plt.plot(t, zscore(mmw_rs), label="mmWave (z)", alpha=0.85)
        plt.title("UWB vs mmWave (normalized)")
        plt.xlabel("Time (s)"); plt.ylabel("Amplitude"); plt.legend(); plt.tight_layout()

        # cross-correlation
        plt.figure(figsize=(10,4))
        plt.plot(lags/target_fs, corr); plt.axvline(lag_seconds, ls="--")
        plt.title("Cross-correlation (peak = estimated lag)")
        plt.xlabel("Lag (s)  [positive = mmWave later]"); plt.ylabel("xcorr"); plt.tight_layout()

        # overlay after shift
        mmw_aligned = np.roll(mmw_rs, -lag_samples).astype(float)
        if lag_samples > 0:   # we shifted left; invalidate right-wrap
            mmw_aligned[-lag_samples:] = np.nan
        elif lag_samples < 0:
            mmw_aligned[:-lag_samples] = np.nan

        plt.figure(figsize=(10,4))
        plt.plot(t, uwb_rs, label="UWB (ref)")
        plt.plot(t, mmw_aligned, label=f"mmWave shifted by {-lag_seconds:.6f} s")
        plt.title("Overlay after applying estimated lag")
        plt.xlabel("Time (s)"); plt.ylabel("Amplitude"); plt.legend(); plt.tight_layout()
        plt.show()

    return {
        "lag_samples": lag_samples,
        "lag_seconds": lag_seconds,
        "target_fs": target_fs,
        "r_no_shift": float(r),
        "r_after_shift": float(r_aligned),
        "N_common": int(L),
    }


In [2]:
compare_uwb_vs_mmw(
    uwb_path="src/data/RVTALL/Processed_cut_data/uwb_processed/1/sentences1/sample_.npy",
    mmw_path="src/data/RVTALL/Processed_cut_data/radar_processed/<set>/<corpus>/sample_123.npy",
    fs_uwb=1000.0,    # replace with real Hz if known
    fs_mmw=1000.0,    # replace with real Hz if known
    prefer_time_axis=1,  # if arrays are (range, time) use 1; switch to 0 if (time, range)
    collapse="rms"
)


FileNotFoundError: [Errno 2] No such file or directory: 'src/data/RVTALL/Processed_cut_data/uwb_processed/<set>/<corpus>/sample_123.npy'